In [1]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time # for sleep time
import csv
import re
import functools
import datetime #for csv file title

driver = webdriver.Chrome(r'chromedriver.exe')

CP=["San juan tepepan", "Whistler"]
for j in range (0, len(CP)):
    driver.get('https://www.airbnb.mx/s/homes?refinement_paths%5B%5D=%2Fhomes&search_type=section_navigation&s_tag=yVNy3gQo')

    element = driver.find_element_by_id("Koan-via-SearchHeader__input")
    city = CP[j]
    element.send_keys(city)
    element.send_keys(Keys.RETURN)
    time.sleep(10)
    
    now = datetime.datetime.now() #for csv file title

    #For csv file names
    def reduce_concat(x, sep=""):
        return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

    def paste(*lists, sep=" ", collapse=None):
        result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
        if collapse is not None:
            return reduce_concat(result, sep=collapse)
        return list(result)

    #Open CSV Writer
    filename = paste(['airBnb_'],[city], [now.year], ['-'], [now.month], ['-'], [now.day],['.csv'], sep='')
    filename = ''.join(filename)
    csv_file = open(filename, 'w', encoding='utf-8',newline = '')
    writer = csv.writer(csv_file)

    index = 1
    i = 1

    house_all_links = []

    while True:

        if index > 1:
            break
        try:
            print("Getting list of links from Page " + str(index))
            index = index + 1
            #driver.maximize_window()
            time.sleep(10) # give new page time to load

            house_list = (driver.find_elements_by_xpath('//a[@class="_1ol0z3h"]'))

            house_links = [house.get_attribute('href') for house in house_list ] 

            house_all_links.extend(house_links)

            # Locate the next button on the page.
            wait_button = WebDriverWait(driver, 10)

            next_button = wait_button.until(EC.element_to_be_clickable((By.XPATH,'//li[@class="_r4n1gzb"]')))

            next_button.click()


        except Exception as e:
            print(e)
            continue


    for url in house_all_links:

        # Initialize an empty dictionary for each review
        airbnb_dict = {}
        driver.get(url)

        print(url)
        try:
            time.sleep(3)
            try:
                house_title = driver.find_element_by_xpath('//h1/span[@class="_18hrqvin"]').text
            except:
                print("issue in house title")
                house_title = None

            try:
                house_type =  driver.find_element_by_xpath('//div[@class="_1p3joamp"]').text
            except:
                print("issue in house type")
                house_type = None

            time.sleep(5)    

            try:

                price = driver.find_element_by_xpath('//span/span[@class="_doc79r"]').text
            except:
                print("issue in price")
                price = None        

            try:
                user = driver.find_element_by_xpath('//div[@class="_8b6uza1"]').text
            except:
                print("issue in user")
                user = None  

            try:
                no_of_ratings = driver.find_element_by_xpath('//div[@class="_vy3ibx"]/h2/span').text

            except:
                print("issue in no_of_ratings")
                no_of_ratings = None  

            time.sleep(3.5)    
            try:
                rating = driver.find_element_by_xpath('//div[@class="_l0ao8q"]//div[@itemprop="ratingValue"]').get_attribute('content')

            except:
                print("issue in rating")
                rating = None

            time.sleep(2)        
            try:
                max_occupancy = driver.find_elements_by_xpath('//div/div[@class="_36rlri"]/div[@class="_czm8crp"]')

                house_detail = [detail.text for detail in max_occupancy ]

            except:
                print("Issue in occupncy")
                house_detail = None

            airbnb_dict['city'] = city
            airbnb_dict['house_title'] = house_title
            airbnb_dict['house_type'] = house_type
            airbnb_dict['price'] = price
            airbnb_dict['user'] = user
            airbnb_dict['no_of_reviews'] = no_of_ratings
            airbnb_dict['rating'] = rating
            airbnb_dict['house_detail'] = house_detail

            print("writing")
            writer.writerow(airbnb_dict.values())

            print(i)
            print(house_title)
            print(house_type)
            print(price)
            print(user)
            print(no_of_ratings)
            print(rating)
            print(house_detail)
            i+= 1

        except Exception as e:
            print(e)
            continue

    csv_file.close()
    #driver.close()
driver.close()

Getting list of links from Page 1
https://www.airbnb.mx/rooms/28226441?location=San%20juan%20tepepan&previous_page_section_name=1000
writing
1
Casa particular CDMX, habitacion privada.
Habitación privada en casa
$202
Adriana
5 evaluaciones
5
['2 huéspedes', '1 recámara', '2 camas', '1 baño entero y uno de tocador compartidos', '\U000f0004\n7 evaluaciones']
https://www.airbnb.mx/rooms/33551076?location=San%20juan%20tepepan&previous_page_section_name=1000
writing
2
Xochimilco al mejor estilo, disfruta aquí.
Habitación privada en casa
$240
Norma
17 evaluaciones
5
['3 huéspedes', '2 recámaras', '2 camas', '2 baños', '\U000f0004\n17 evaluaciones']
https://www.airbnb.mx/rooms/35914614?location=San%20juan%20tepepan&previous_page_section_name=1000
issue in no_of_ratings
issue in rating
writing
3
Comodo alojamiento para estudiantes
Habitación privada en departamento
$195
José Edgar Germán
None
None
['1 huésped', '1 recámara', '1 cama', '1 baño compartido']
https://www.airbnb.mx/rooms/31622231?l

issue in no_of_ratings
writing
9
Cozy Whistler Village Studio
Condominio completo
$1,300
Tim And Andrea
None
5
['4 huéspedes', 'estudio', '2 camas', '1 baño', '\U000f0004\n484 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/18074253?location=Whistler&previous_page_section_name=1000
issue in no_of_ratings
writing
10
Symphony Studio in Whistler Village
Departamento entero
$1,753
Musubi
None
5
['4 huéspedes', 'estudio', '2 camas', '1 baño', '\U000f0004\n194 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/9670429?location=Whistler&previous_page_section_name=1000
issue in no_of_ratings
writing
11
⛰️ 1 Bdrm Modern Village Penthouse - view!
Departamento con servicios completo
$2,760
Victoria And Christian
None
5
['4 huéspedes', '1 recámara', '2 camas', '1 baño', '\U000f0004\n1167 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/4249398?location=Whistler&previous_page_section_name=1000
issue in no_of_ratings
writing
12
Ski in/out co

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time # for sleep time
import csv
import re
import functools
import datetime #for csv file title

driver = webdriver.Chrome(r'chromedriver.exe')

#driver.get('https://www.airbnb.mx/s/Whistler--Columbia-Brit%C3%A1nica--Canad%C3%A1/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&query=Whistler%2C%20Columbia%20Brit%C3%A1nica%2C%20Canad%C3%A1&place_id=ChIJh3uVA7I8h1QR9s_1dehBt0o&search_type=filter_change&adults=1&s_tag=fxn88d36')
driver.get('https://www.airbnb.mx/s/homes?refinement_paths%5B%5D=%2Fhomes&search_type=section_navigation&s_tag=yVNy3gQo')

element = driver.find_element_by_id("Koan-via-SearchHeader__input")
city = "Whistler"
element.send_keys(city)
element.send_keys(Keys.RETURN)

now = datetime.datetime.now() #for csv file title

#For csv file names
def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)

#Open CSV Writer
filename = paste(['airBnb_Whistler'], [now.year], ['-'], [now.month], ['-'], [now.day],['.csv'], sep='')
filename = ''.join(filename)
csv_file = open(filename, 'w', encoding='utf-8',newline = '')
writer = csv.writer(csv_file)

index = 1
i = 1

house_all_links = []

while True:
    
    if index > 1:
        break
    try:
        print("Getting list of links from Page " + str(index))
        index = index + 1
        #driver.maximize_window()
        time.sleep(10) # give new page time to load
                
        house_list = (driver.find_elements_by_xpath('//a[@class="_1ol0z3h"]'))

        house_links = [house.get_attribute('href') for house in house_list ] 
        
        house_all_links.extend(house_links)
  
        # Locate the next button on the page.
        wait_button = WebDriverWait(driver, 10)
        
        next_button = wait_button.until(EC.element_to_be_clickable((By.XPATH,'//li[@class="_r4n1gzb"]')))
        
        next_button.click()

        
    except Exception as e:
        print(e)
        continue
        

for url in house_all_links:

    # Initialize an empty dictionary for each review
    airbnb_dict = {}
    driver.get(url)

    print(url)
    try:
        time.sleep(3)
        try:
            house_title = driver.find_element_by_xpath('//h1/span[@class="_18hrqvin"]').text
        except:
            print("issue in house title")
            house_title = None

        try:
            house_type =  driver.find_element_by_xpath('//div[@class="_1p3joamp"]').text
        except:
            print("issue in house type")
            house_type = None
            
        time.sleep(5)    

        try:
            
            price = driver.find_element_by_xpath('//span/span[@class="_doc79r"]').text
        except:
            print("issue in price")
            price = None        

        try:
            user = driver.find_element_by_xpath('//div[@class="_8b6uza1"]').text
        except:
            print("issue in user")
            user = None  

        try:
            no_of_ratings = driver.find_element_by_xpath('//div[@class="_vy3ibx"]/h2/span').text

        except:
            print("issue in no_of_ratings")
            no_of_ratings = None  
        
        time.sleep(3.5)    
        try:
            rating = driver.find_element_by_xpath('//div[@class="_l0ao8q"]//div[@itemprop="ratingValue"]').get_attribute('content')

        except:
            print("issue in rating")
            rating = None

        time.sleep(2)        
        try:
            max_occupancy = driver.find_elements_by_xpath('//div/div[@class="_36rlri"]/div[@class="_czm8crp"]')

            house_detail = [detail.text for detail in max_occupancy ]

        except:
            print("Issue in occupncy")
            house_detail = None

        airbnb_dict['city'] = city
        airbnb_dict['house_title'] = house_title
        airbnb_dict['house_type'] = house_type
        airbnb_dict['price'] = price
        airbnb_dict['user'] = user
        airbnb_dict['no_of_reviews'] = no_of_ratings
        airbnb_dict['rating'] = rating
        airbnb_dict['house_detail'] = house_detail

        print("writing")
        writer.writerow(airbnb_dict.values())

        print(i)
        print(house_title)
        print(house_type)
        print(price)
        print(user)
        print(no_of_ratings)
        print(rating)
        print(house_detail)
        i+= 1

    except Exception as e:
        print(e)
        continue
        
csv_file.close()
driver.close()

In [5]:
cols =["city","house_title","house_type","price","owner_name","no_of_reviews","rating","occupancy"]

#read in the csv file
airbnbdf = pd.read_csv('airBnb_Whistler2019-7-18.csv',encoding = 'utf-8', names=cols)
airbnbdf.head(5)

,city,house_title,house_type,price,owner_name,no_of_reviews,rating,occupancy
0,Whistler,Newly updated condo at Blackcomb base,Condominio completo,"$1,460",Caroline,119 evaluaciones,4.5,"['4 huéspedes', '1 recámara', '1 cama', '1 bañ..."
1,Whistler,Peak View Penthouse,Loft entero,"$1,884",Jane & James,158 evaluaciones,5.0,"['4 huéspedes', '1 recámara', '1 cama', '1 bañ..."
2,Whistler,1 BR/Bonus Loft/Wood fireplace/NETFLIX/washerd...,Condominio completo,"$1,183",Sophie,131 evaluaciones,5.0,"['4 huéspedes', '1 recámara', '3 camas', '1 ba..."
3,Whistler,Symphony Studio in Whistler Village,Departamento entero,"$1,753",Musubi,194 evaluaciones,5.0,"['4 huéspedes', 'estudio', '2 camas', '1 baño'..."
4,Whistler,"Wolf's Den Creekside, steps from the gondola",Condominio completo,$862,Jane,39 evaluaciones,4.5,"['3 huéspedes', '1 recámara', '2 camas', '1 ba..."


In [6]:
#verify the datatypes
airbnbdf.dtypes

city              object
house_title       object
house_type        object
price             object
owner_name        object
no_of_reviews     object
rating           float64
occupancy         object
dtype: object

In [7]:
airbnbdf.price = airbnbdf.price.astype(str)
airbnbdf.no_of_reviews = airbnbdf.no_of_reviews.astype(str)


def remove_special_chars(val):
    new_val = re.findall('[0-9]',val)
    return "".join(new_val)

airbnbdf.price = airbnbdf.price.apply(remove_special_chars)
print(airbnbdf.price.head(5))


0    1460
1    1884
2    1183
3    1753
4     862
Name: price, dtype: object


In [8]:

airbnbdf.no_of_reviews = airbnbdf.no_of_reviews.apply(remove_special_chars)
print(airbnbdf.no_of_reviews.head(5))


0    119
1    158
2    131
3    194
4     39
Name: no_of_reviews, dtype: object


In [9]:
def split_to_list(str_val):
    return str_val.split(",")
    

airbnbdf['occupancy_lst'] = airbnbdf.occupancy.apply(split_to_list)

airbnbdf.occupancy_lst.sample(5)

14    [['5 huéspedes',  '2 recámaras',  '3 camas',  ...
3     [['4 huéspedes',  'estudio',  '2 camas',  '1 b...
12    [['2 huéspedes',  'estudio',  '1 cama',  '1 ba...
7     [['4 huéspedes',  'estudio',  '2 camas',  '1 b...
11    [['2 huéspedes',  '1 recámara',  '1 cama',  '1...
Name: occupancy_lst, dtype: object

In [10]:
len(airbnbdf)

airbnbdf.occupancy_lst[0][0]


"['4 huéspedes'"

In [11]:
airbnbdf = airbnbdf.assign(no_of_guest=airbnbdf.price.mean(), house_size=airbnbdf.price.mean(), no_of_beds = airbnbdf.price.mean(), no_of_baths=airbnbdf.price.mean())

for i in range(len(airbnbdf)):
    try:

        airbnbdf.no_of_guest[i] = airbnbdf.occupancy_lst[i][0]
        airbnbdf.house_size[i]  = airbnbdf.occupancy_lst[i][1]
        airbnbdf.no_of_beds[i]  = airbnbdf.occupancy_lst[i][2]
        airbnbdf.no_of_baths[i]  = airbnbdf.occupancy_lst[i][3]
        
    except:
        continue

In [12]:
airbnbdf.no_of_guest = airbnbdf.no_of_guest.apply(remove_special_chars)
print(airbnbdf.no_of_guest.head(5))

0    4
1    4
2    4
3    4
4    3
Name: no_of_guest, dtype: object


In [13]:
print(sum(airbnbdf.house_size.isna()))

print(sum(airbnbdf.house_size.isnull()))

0
0


In [14]:
def convert_chars(val):
   ##estudio cuenta como 0.5 de un cuarto
    if val == " 'estudio'":
        new_val = '0.5'
    else:
        new_val = re.findall('[0-9]',val)
    
    return "".join(new_val)

airbnbdf.house_size = airbnbdf.house_size.astype(str)
airbnbdf = airbnbdf.assign(rooms=airbnbdf.house_size.apply(convert_chars))

airbnbdf.sample(14)

,city,house_title,house_type,price,owner_name,no_of_reviews,rating,occupancy,occupancy_lst,no_of_guest,house_size,no_of_beds,no_of_baths,rooms
6,Whistler,1 BR Townhome with Private Hot Tub - FREE PARKING,Casa adosada completa,2030,Ian,311,4.5,"['4 huéspedes', '1 recámara', '2 camas', '1 ba...","[['4 huéspedes', '1 recámara', '2 camas', '...",4,'1 recámara','2 camas','1 baño',1
0,Whistler,Newly updated condo at Blackcomb base,Condominio completo,1460,Caroline,119,4.5,"['4 huéspedes', '1 recámara', '1 cama', '1 bañ...","[['4 huéspedes', '1 recámara', '1 cama', '1...",4,'1 recámara','1 cama','1 baño',1
12,Whistler,˙Studio Condo Whistler Village 223,Departamento entero,1241,Bill,392,5.0,"['2 huéspedes', 'estudio', '1 cama', '1 baño',...","[['2 huéspedes', 'estudio', '1 cama', '1 ba...",2,'estudio','1 cama','1 baño',0.5
16,Whistler,"The Mini Chalet - Steps to Gondola, Lakes & Tr...",Casa adosada completa,1446,Jodie & Ulla,103,5.0,"['4 huéspedes', '1 recámara', '2 camas', '1 ba...","[['4 huéspedes', '1 recámara', '2 camas', '...",4,'1 recámara','2 camas','1 baño',1
14,Whistler,"Prime location, 2Br/2Ba, with private HotTub &...",Condominio completo,1533,Karel,68,5.0,"['5 huéspedes', '2 recámaras', '3 camas', '2 b...","[['5 huéspedes', '2 recámaras', '3 camas', ...",5,'2 recámaras','3 camas','2 baños',2
3,Whistler,Symphony Studio in Whistler Village,Departamento entero,1753,Musubi,194,5.0,"['4 huéspedes', 'estudio', '2 camas', '1 baño'...","[['4 huéspedes', 'estudio', '2 camas', '1 b...",4,'estudio','2 camas','1 baño',0.5
8,Whistler,Top-Floor 1BR Condo in Whistler Village w/ Par...,Departamento entero,1753,Pranjal,188,5.0,"['4 huéspedes', '1 recámara', '2 camas', '1 ba...","[['4 huéspedes', '1 recámara', '2 camas', '...",4,'1 recámara','2 camas','1 baño',1
13,Whistler,Cozy studio on Main Street with hot tub and pool,Condominio completo,1533,Jenny And Roman,210,5.0,"['2 huéspedes', 'estudio', '1 cama', '1 baño',...","[['2 huéspedes', 'estudio', '1 cama', '1 ba...",2,'estudio','1 cama','1 baño',0.5
15,Whistler,Quiet Mountainside Escape: Ski/Bike In-Out,Condominio completo,1241,John & Hilary,118,5.0,"['4 huéspedes', '1 recámara', '2 camas', '1 ba...","[['4 huéspedes', '1 recámara', '2 camas', '...",4,'1 recámara','2 camas','1 baño',1
5,Whistler,Ski in/out condo in Whistler with hot tub/pool,Departamento entero,1753,Claudia And Eli,40,5.0,"['4 huéspedes', '1 recámara', '3 camas', '1 ba...","[['4 huéspedes', '1 recámara', '3 camas', '...",4,'1 recámara','3 camas','1 baño',1


In [15]:
def remove_chars(val):
    new_val = re.findall('[0-9.]+',val)
    return "".join(new_val)


airbnbdf.no_of_beds = airbnbdf.no_of_beds.astype(str)
airbnbdf.no_of_baths = airbnbdf.no_of_baths.astype(str)

airbnbdf = airbnbdf.assign(beds=airbnbdf.no_of_beds.apply(remove_chars))
airbnbdf = airbnbdf.assign(bath=airbnbdf.no_of_baths.apply(remove_chars))
airbnbdf.sample(4)

,city,house_title,house_type,price,owner_name,no_of_reviews,rating,occupancy,occupancy_lst,no_of_guest,house_size,no_of_beds,no_of_baths,rooms,beds,bath
5,Whistler,Ski in/out condo in Whistler with hot tub/pool,Departamento entero,1753,Claudia And Eli,40,5.0,"['4 huéspedes', '1 recámara', '3 camas', '1 ba...","[['4 huéspedes', '1 recámara', '3 camas', '...",4,'1 recámara','3 camas','1 baño',1,3,1
2,Whistler,1 BR/Bonus Loft/Wood fireplace/NETFLIX/washerd...,Condominio completo,1183,Sophie,131,5.0,"['4 huéspedes', '1 recámara', '3 camas', '1 ba...","[['4 huéspedes', '1 recámara', '3 camas', '...",4,'1 recámara','3 camas','1 baño',1,3,1
9,Whistler,1 BR Townhouse with Private Hot Tub - FREE PAR...,Casa adosada completa,1884,Ian,317,4.5,"['5 huéspedes', '1 recámara', '3 camas', '1 ba...","[['5 huéspedes', '1 recámara', '3 camas', '...",5,'1 recámara','3 camas','1 baño',1,3,1
3,Whistler,Symphony Studio in Whistler Village,Departamento entero,1753,Musubi,194,5.0,"['4 huéspedes', 'estudio', '2 camas', '1 baño'...","[['4 huéspedes', 'estudio', '2 camas', '1 b...",4,'estudio','2 camas','1 baño',0.5,2,1


In [16]:
(airbnbdf == "none").any() #check for any none values

city             False
house_title      False
house_type       False
price            False
owner_name       False
no_of_reviews    False
rating           False
occupancy        False
occupancy_lst    False
no_of_guest      False
house_size       False
no_of_beds       False
no_of_baths      False
rooms            False
beds             False
bath             False
dtype: bool

In [17]:
(airbnbdf.isnull()).any()

city             False
house_title      False
house_type       False
price            False
owner_name       False
no_of_reviews    False
rating           False
occupancy        False
occupancy_lst    False
no_of_guest      False
house_size       False
no_of_beds       False
no_of_baths      False
rooms            False
beds             False
bath             False
dtype: bool

In [18]:
(airbnbdf.isna()).any()

city             False
house_title      False
house_type       False
price            False
owner_name       False
no_of_reviews    False
rating           False
occupancy        False
occupancy_lst    False
no_of_guest      False
house_size       False
no_of_beds       False
no_of_baths      False
rooms            False
beds             False
bath             False
dtype: bool

In [19]:
print(len(airbnbdf))
airbnbdf.isna().sum().sum() #check num of Nans

18


0

In [20]:
airbnbdf.dropna() #check 

,city,house_title,house_type,price,owner_name,no_of_reviews,rating,occupancy,occupancy_lst,no_of_guest,house_size,no_of_beds,no_of_baths,rooms,beds,bath
0,Whistler,Newly updated condo at Blackcomb base,Condominio completo,1460,Caroline,119,4.5,"['4 huéspedes', '1 recámara', '1 cama', '1 bañ...","[['4 huéspedes', '1 recámara', '1 cama', '1...",4,'1 recámara','1 cama','1 baño',1,1,1
1,Whistler,Peak View Penthouse,Loft entero,1884,Jane & James,158,5.0,"['4 huéspedes', '1 recámara', '1 cama', '1 bañ...","[['4 huéspedes', '1 recámara', '1 cama', '1...",4,'1 recámara','1 cama','1 baño',1,1,1
2,Whistler,1 BR/Bonus Loft/Wood fireplace/NETFLIX/washerd...,Condominio completo,1183,Sophie,131,5.0,"['4 huéspedes', '1 recámara', '3 camas', '1 ba...","[['4 huéspedes', '1 recámara', '3 camas', '...",4,'1 recámara','3 camas','1 baño',1,3,1
3,Whistler,Symphony Studio in Whistler Village,Departamento entero,1753,Musubi,194,5.0,"['4 huéspedes', 'estudio', '2 camas', '1 baño'...","[['4 huéspedes', 'estudio', '2 camas', '1 b...",4,'estudio','2 camas','1 baño',0.5,2,1
4,Whistler,"Wolf's Den Creekside, steps from the gondola",Condominio completo,862,Jane,39,4.5,"['3 huéspedes', '1 recámara', '2 camas', '1 ba...","[['3 huéspedes', '1 recámara', '2 camas', '...",3,'1 recámara','2 camas','1 baño',1,2,1
5,Whistler,Ski in/out condo in Whistler with hot tub/pool,Departamento entero,1753,Claudia And Eli,40,5.0,"['4 huéspedes', '1 recámara', '3 camas', '1 ba...","[['4 huéspedes', '1 recámara', '3 camas', '...",4,'1 recámara','3 camas','1 baño',1,3,1
6,Whistler,1 BR Townhome with Private Hot Tub - FREE PARKING,Casa adosada completa,2030,Ian,311,4.5,"['4 huéspedes', '1 recámara', '2 camas', '1 ba...","[['4 huéspedes', '1 recámara', '2 camas', '...",4,'1 recámara','2 camas','1 baño',1,2,1
7,Whistler,Cozy Whistler Village Studio,Condominio completo,1300,Tim And Andrea,308,5.0,"['4 huéspedes', 'estudio', '2 camas', '1 baño'...","[['4 huéspedes', 'estudio', '2 camas', '1 b...",4,'estudio','2 camas','1 baño',0.5,2,1
8,Whistler,Top-Floor 1BR Condo in Whistler Village w/ Par...,Departamento entero,1753,Pranjal,188,5.0,"['4 huéspedes', '1 recámara', '2 camas', '1 ba...","[['4 huéspedes', '1 recámara', '2 camas', '...",4,'1 recámara','2 camas','1 baño',1,2,1
9,Whistler,1 BR Townhouse with Private Hot Tub - FREE PAR...,Casa adosada completa,1884,Ian,317,4.5,"['5 huéspedes', '1 recámara', '3 camas', '1 ba...","[['5 huéspedes', '1 recámara', '3 camas', '...",5,'1 recámara','3 camas','1 baño',1,3,1


In [21]:
airbnbdfClean = airbnbdf.dropna() #drop all NaN's

In [22]:
print(len(airbnbdfClean))
airbnbdfClean.isna().sum().sum() #check num of Nans

18


0

In [23]:
airbnbdfClean["price"].replace("",0, inplace=True)
airbnbdfClean["no_of_guest"].replace("",0, inplace=True)
airbnbdfClean["rooms"].replace("",0, inplace=True)
airbnbdfClean["beds"].replace("",0, inplace=True)
airbnbdfClean["bath"].replace("",0, inplace=True)

In [24]:
#convert price to float
airbnbdfClean.price=airbnbdfClean.price.astype(float)

#convert no_of_reviews to float
airbnbdfClean.no_of_reviews=airbnbdfClean.no_of_reviews.astype(int)

#convert no_of_guest to float
airbnbdfClean.no_of_guest=airbnbdfClean.no_of_guest.astype(float)

#convert house to float
airbnbdfClean.rooms=airbnbdfClean.rooms.astype(float)

#convert house to float
airbnbdfClean.beds=airbnbdfClean.beds.astype(float)

#convert house to float
airbnbdfClean.bath=airbnbdfClean.bath.astype(float)

In [25]:
del airbnbdfClean["occupancy"]
del airbnbdfClean["occupancy_lst"]

In [26]:
airbnbdfClean.sample(5)

,city,house_title,house_type,price,owner_name,no_of_reviews,rating,no_of_guest,house_size,no_of_beds,no_of_baths,rooms,beds,bath
17,Whistler,Hidden Gem of Whistler Village.,Casa adosada completa,2264.0,Dany,41,5.0,6.0,'2 recámaras','2 camas','2 baños',2.0,2.0,2.0
7,Whistler,Cozy Whistler Village Studio,Condominio completo,1300.0,Tim And Andrea,308,5.0,4.0,'estudio','2 camas','1 baño',0.5,2.0,1.0
6,Whistler,1 BR Townhome with Private Hot Tub - FREE PARKING,Casa adosada completa,2030.0,Ian,311,4.5,4.0,'1 recámara','2 camas','1 baño',1.0,2.0,1.0
2,Whistler,1 BR/Bonus Loft/Wood fireplace/NETFLIX/washerd...,Condominio completo,1183.0,Sophie,131,5.0,4.0,'1 recámara','3 camas','1 baño',1.0,3.0,1.0
4,Whistler,"Wolf's Den Creekside, steps from the gondola",Condominio completo,862.0,Jane,39,4.5,3.0,'1 recámara','2 camas','1 baño',1.0,2.0,1.0


In [98]:
# write the cleaned and massaged data to csv for future analysis

airbnbdfClean.to_csv('airbnbdfClean.csv', index=False)